In [5]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [ ]:
%pip install langchain_google_community langchain_community langchain_google_genai langgraph langchain_text_splitters 

In [ ]:
%pip install beautifulsoup4 faiss-cpu

In [73]:
from dotenv import load_dotenv
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

def rag_search(
    query: str,
    search_results_num: int = 10,
    context_top_k: int = 5,
    chunk_size: int = 100,
    chunk_overlap: int = 50,
    embedding_model: str = "models/embedding-001",
    llm_model: str = "gemini-2.0-flash-001"
):
    """
    query: 질의 문자열
    search_results_num: 구글 검색해서 가져올 결과 문서 개수
    context_top_k: context로 넘길 벡터DB Top-k 개수
    chunk_size, chunk_overlap: 청크 분할 길이
    embedding_model, llm_model: 사용할 구글 임베딩/LLM 모델명
    """

    # 1. 환경 변수 및 검색
    load_dotenv()
    search = GoogleSearchAPIWrapper()
    results = search.results(query, search_results_num)

    docs = []
    for result in results:
        text = result['title'] + ' ' + result.get('snippet', '')
        doc = Document(page_content=text, metadata=result)
        docs.append(doc)

    # 2. 텍스트 쪼개기
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    split_documents = text_splitter.split_documents(docs)

    # 3. 벡터스토어 생성
    embeddings = GoogleGenerativeAIEmbeddings(model=embedding_model)
    vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

    # 4. 검색기(리트리버) 생성
    retriever = vectorstore.as_retriever(search_kwargs={"k": context_top_k})

    # 5. 프롬프트 및 RAG chain 생성
    prompt = PromptTemplate.from_template(
        """당신은 전문적이고 신뢰할 수 있는 AI 어시스턴트입니다.
제가 제시하는 개념, 용어 또는 문장에 대해 그 정의와 원리, 배경을 중심으로 설명해 주세요.
또한 해당 개념이 실제로 어떻게 적용되는지, 실생활이나 다양한 분야에서의 예시를 들어주면 좋겠습니다.
모든 설명은 비전문가도 쉽게 이해할 수 있도록 차근차근 자세하게 풀어서 설명해 주세요.

#Context:
{context}

#Question:
{question}

#Answer:"""
    )

    llm = ChatGoogleGenerativeAI(model=llm_model, temperature=0)

    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 6. 체인 실행
    result = rag_chain.invoke(query)
    return result


# 사용 예시
if __name__ == "__main__":
    answer = rag_search(
        query="what is the AI Agent",
        search_results_num=8,      # 구글 결과 8개
        context_top_k=3            # Top-3 문서 context로 사용
    )
    print(answer)


## AI 에이전트 (AI Agent) 란 무엇일까요?

AI 에이전트는 한마디로 **"특정 목표를 달성하기 위해 스스로 판단하고 행동하는 똑똑한 소프트웨어 시스템"** 이라고 할 수 있습니다. 마치 사람처럼 주어진 상황을 인식하고, 문제를 해결하기 위한 계획을 세우고, 필요한 도구를 활용하여 목표를 달성하는 것이죠.

**좀 더 자세히 풀어볼까요?**

*   **정의:** AI 에이전트는 인공지능 기술을 기반으로 만들어진 자율적인 소프트웨어 시스템입니다. 이 시스템은 주어진 환경을 인식하고, 목표를 설정하고, 그 목표를 달성하기 위해 스스로 판단하고 행동합니다.
*   **원리:** AI 에이전트는 주로 다음과 같은 원리로 작동합니다.
    *   **인식 (Perception):** 센서나 API 등을 통해 외부 환경으로부터 정보를 수집합니다. 예를 들어, 카메라를 통해 이미지를 인식하거나, 웹 API를 통해 날씨 정보를 얻는 것이죠.
    *   **추론 (Reasoning):** 수집된 정보를 바탕으로 현재 상황을 분석하고, 목표 달성을 위한 최적의 행동 계획을 세웁니다. 이때 다양한 인공지능 알고리즘 (예: 머신러닝, 딥러닝)이 활용됩니다.
    *   **행동 (Action):** 계획에 따라 실제로 행동을 수행합니다. 예를 들어, 로봇 팔을 움직여 물건을 옮기거나, 이메일을 보내는 것이죠.
    *   **학습 (Learning):** 행동의 결과를 평가하고, 다음번에는 더 나은 결과를 얻을 수 있도록 스스로 학습합니다.
*   **배경:** AI 에이전트의 개념은 오래전부터 인공지능 연구 분야에서 논의되어 왔습니다. 초기에는 단순한 규칙 기반의 에이전트가 개발되었지만, 최근에는 머신러닝과 딥러닝 기술의 발전 덕분에 훨씬 더 복잡하고 지능적인 AI 에이전트가 등장하고 있습니다.

**AI 에이전트는 실제로 어떻게 적용될까요?**

AI 에이전트는 다양한 분야에서 활용될 수 있습니다. 몇 가지 예시를 살펴볼까요?

*   **가상 비서 (V